# S<sub>N</sub>2 Reaction

This notebook introduces some basic concepts for the use of Asparagus Bundle in the context of a reactive potential.

The reaction to simulate is a basic S<sub>N</sub>2 reaction between a methyl bromide and a hydroxide ion. The reaction is shown below.

CH<sub>3</sub>Br + OH<sup>-</sup> -> CH<sub>3</sub>OH + Br<sup>-</sup>

We will generate a reactive potential energy surface for this reaction using xtb.

Although the reaction looks very simple, it actually involves multiple channels. The reaction can proceed via a frontside or backside attack of the hydroxide ion. Therefore, both chanels should be considered in the simulation.
A schematic representation of the PES is shown below.

![sn2_scheme](Sn2_scheme.gif)

Adapted from: Tasi, D. A., Fábián, Z., & Czakó, G. (2019). Rethinking the X−+ CH 3 Y [X= OH, SH, CN, NH 2, PH 2; Y= F, Cl, Br, I] SN 2 reactions. Physical Chemistry Chemical Physics, 21(15), 7924-7931.

The third channel called that involves the extration of the proton from the methyl group is not considered in this tutorial because it is very high in energy.

In [3]:
# Import basic libraries
import numpy as np
import matplotlib.pyplot as plt # for plotting
# ASE importantions for manipulating structures
import ase
from ase.io import read, write #Read and write structures
from ase.visualize import view #Visualize structures

Tasi *et al.* have already performed the calculations for the reaction at a very high level of theory (CCSD(T)-F12/AVTZ). The structures are available in the `sn2_struct` folder. The structures are in xyz format. We can read the structures using ASE. First, we check the structure of the first channel which involves the attack of the hydroxide ion from the frontside of the halogen atom.

In [2]:
file = 'Sn2_struct/fs_min.xyz'
fs = read(file)

In [3]:
view(fs,viewer='x3d')


Next we can check the structure of the second channel which involves the attack of the hydroxide ion from the backside of the halogen atom. This is the most known channel for the S<sub>N</sub>2 reactions.

In [3]:
file = 'Sn2_struct/pre_min.xyz'
pre = read(file)

In [4]:
view(pre,viewer='x3d')

Now, that we know both of the initial intermediate states, we will run a metadynamics simulations to generate the first part of the database that we will use to train our model. 

In [5]:
# Import the asparagus bundle
from asparagus import Asparagus # Import the asparagus bundle package
from asparagus import MDSampler, NormalModeScanner, MetaSampler # Import the different sampling methods

#Scan the angle around the C-Br-O bond
sampler = MetaSampler(
        config='sn2_config.json',
        sample_directory='sn2_samples',
        sample_systems=['Sn2_struct/fs_min.xyz'],
        sample_systems_format='xyz',
        sample_calculator='XTB',
        sample_properties=['energy', 'forces', 'dipole'],
        sample_systems_optimize=False,
        meta_cv=[[0,2,1]],
        meta_hookean=[[0,2,1,6.0]],
        meta_gaussian_height=0.10,
        meta_gaussian_widths=0.2,
        meta_gaussian_interval=10,
        meta_time_step=1.0,
        meta_simulation_time=3000.0,
        meta_save_interval=10,
        meta_temperature=300,
        meta_langevin_friction=1.0,
        meta_initial_velocities=True,
        meta_initial_temperature=300.,
    )
sampler.run()


INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Adding parameter 'sample_data_file'.
Conflict! Overwrite parameter 'sample_directory'.
Conflict! Overwrite parameter 'sample_systems'.
Conflict! Overwrite parameter 'sample_systems_format'.
Conflict! Overwrite parameter 'sample_calculator'.
Conflict! Overwrite parameter 'sample_calculator_args'.
Conflict! Overwrite parameter 'sample_properties'.
Conflict! Overwrite parameter 'sample_systems_optimize'.
Conflict! Overwrite parameter 'sample_systems_optimize_fmax'.
Conflict! Overwrite parameter 'sample_data_overwrite'.
Conflict! Overwrite parameter 'sample_tag'.



The previous block start the sampling module and write a new configuration file called `sn2_config.json`. This file contains the information about the sampling. In the first part, we only sample the 

In [7]:
# Look at the trajectory generated
from ase.io import Trajectory
traj = Trajectory('sn2_samples/1_meta.traj')
view(traj)

In [8]:
from asparagus import MDSampler, NormalModeScanner, MetaSampler # Import the different sampling methods
sampler = MetaSampler(
        config='sn2_config.json',
        sample_directory='sn2_samples',
        sample_systems=['Sn2_struct/wal_ts.xyz'],
        sample_systems_format='xyz',
        sample_calculator='XTB',
        sample_properties=['energy', 'forces', 'dipole'],
        sample_systems_optimize=False,
        meta_cv=[[0,2]],
        meta_hookean=[[0,2,5.0]],
        meta_gaussian_height=0.2,
        meta_gaussian_widths=0.2,
        meta_gaussian_interval=10,
        meta_time_step=1.0,
        meta_simulation_time=500.0,
        meta_save_interval=10,
        meta_temperature=300,
        meta_langevin_friction=1.0,
        meta_initial_velocities=True,
        meta_initial_temperature=300.,
    )
sampler.run()

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Adding parameter 'sample_data_file'.
Conflict! Overwrite parameter 'sample_directory'.
Conflict! Overwrite parameter 'sample_systems'.
Conflict! Overwrite parameter 'sample_systems_format'.
Conflict! Overwrite parameter 'sample_calculator'.
Conflict! Overwrite parameter 'sample_calculator_args'.
Conflict! Overwrite parameter 'sample_properties'.
Conflict! Overwrite parameter 'sample_systems_optimize'.
Conflict! Overwrite parameter 'sample_systems_optimize_fmax'.
Conflict! Overwrite parameter 'sample_data_overwrite'.
Conflict! Overwrite parameter 'sample_tag'.



In [16]:
traj2 = Trajectory('sn2_samples/2_meta.traj')
view(traj2)

NameError: name 'Trajectory' is not defined

Now we sample over products and reactants with Normal Mode Sampling

In [1]:
from asparagus import NormalModeSampler
sampler_nms = NormalModeSampler( config='sn2_config.json',
                                 sample_directory='sn2_samples',
                                 sample_data_file='meta.db',
                                 sample_systems=['Sn2_struct/prod.xyz','Sn2_struct/react.xyz'],
                                 sample_systems_format='xyz',
                                 sample_calculator='XTB',
                                 sample_properties=['energy', 'forces', 'dipole'],
                                 sample_systems_optimize=True,
                                 nms_temperature=300,
                                 nms_nsamples=1000,)
sampler_nms.run()


/home/vazquez/miniforge3/envs/asparagus/lib/python3.8/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "
INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'sample_data_file'.
Conflict! Overwrite parameter 'sample_directory'.
Conflict! Overwrite parameter 'sample_systems'.
Conflict! Overwrite parameter 'sample_systems_format'.
Conflict! Overwrite parameter 'sample_calculator'.
Conflict! Overwrite parameter 'sample_calculator_args'.
Conflict! Overwrite parameter 'sample_properties'.
Conflict! Overwrite parameter 'sample_systems_optimize'.
Conflict! Overwrite parameter 'sample_systems_optimize_fmax'.
Conflict! Overwrite par

---------------------
  #    meV     cm^-1
---------------------
  0    1.3i     10.2i
  1    0.5i      4.0i
  2    0.3i      2.6i
  3    0.0i      0.2i
  4    0.0       0.1
  5    0.4       3.3
  6   10.0      80.4
  7   13.8     111.4
  8   18.7     150.9
  9   56.3     454.2
 10  127.3    1026.9
 11  133.9    1079.9
 12  138.9    1120.7
 13  159.1    1283.6
 14  174.2    1405.2
 15  181.1    1460.4
 16  182.0    1467.7
 17  363.1    2928.8
 18  366.0    2951.8
 19  376.9    3039.8
 20  436.3    3518.6
---------------------
Zero-point energy: 1.369 eV
---------------------
  #    meV     cm^-1
---------------------
  0    0.8i      6.3i
  1    0.5i      3.6i
  2    0.3i      2.4i
  3    0.0i      0.3i
  4    0.0       0.2
  5    0.7       6.0
  6    6.7      54.4
  7    7.5      60.7
  8   17.8     143.2
  9   24.4     196.5
 10   28.6     230.7
 11   48.6     392.0
 12   71.7     578.5
 13   87.8     707.9
 14  116.0     935.9
 15  164.0    1322.7
 16  187.8    1514.8
 17  384.4    

# Training

Now that we finish with the initial sampling, we can train the model, again using asparagus bundle.

In [2]:
from asparagus import Asparagus

model = Asparagus(
    config='sn2_config.json',
    data_file='meta.db',
    model_device='gpu',
    model_directory='sn2_model',
    model_properties=['energy', 'forces', 'dipole'],
    model_interaction_cutoff=8.0,
    trainer_properties_weights={
        'energy': 1.,
        'forces': 50.,
        'dipole': 25.
        },
    trainer_max_epochs=1_000,
    )
model.train()
model.test(
    test_datasets='all',
    test_directory=model.get('model_directory'))

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Adding parameter 'data_file'.
Adding parameter 'model_device'.
Adding parameter 'model_directory'.
Adding parameter 'model_properties'.
Adding parameter 'model_interaction_cutoff'.
Adding parameter 'trainer_properties_weights'.
Adding parameter 'trainer_max_epochs'.

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'data_file'.
Adding parameter 'data_source'.
Adding parameter 'data_format'.
Adding parameter 'data_alt_property_labels'.
Adding parameter 'data_unit_positions'.
Adding parameter 'data_load_properties'.
Adding parameter 'data_unit_properties'.
Adding p


       '   _______                                                  ______                  _ _        
       '  (_______)                                                (____  \                | | |       
       '   _______  ___ ____  _____  ____ _____  ____ _   _  ___    ____)  )_   _ ____   __| | | _____ 
       '  |  ___  |/___)  _ \(____ |/ ___|____ |/ _  | | | |/___)  |  __  (| | | |  _ \ / _  | || ___ |
       '  | |   | |___ | |_| / ___ | |   / ___ ( (_| | |_| |___ |  | |__)  ) |_| | | | ( (_| | || ____|
       '  |_|   |_(___/|  __/\_____|_|   \_____|\___ |____/(___/   |______/|____/|_| |_|\____|\_)_____)
       '               |_|                     (_____|                            
       '
       '                        Authors: K. Toepfer and L.I. Vazquez-Salazar
       '                        Date: 2023-12-20 18:16:00
       '                        Task: Training a Neural Network Potential               
       '                        Running on: pc-mm016 with: 

INFO:asparagus.src.train.tester:INFO:
Summary for test set:
  Property Metrics    MAE,       RMSE
   energy           6.64e-01,  7.63e-01 eV
   forces           3.50e-01,  5.37e-01 eV/Ang
   dipole           2.12e-01,  2.67e-01 eAng

INFO:asparagus.src.train.trainer:INFO:
Summary Epoch: 5/1000
  Loss   train / valid:  8.93E+00 / 8.39E+00  Best Loss valid: 8.39E+00
  Property Metrics (valid):
    energy      MAE (Best) / RMSE (Best):  6.51E-01 (6.51E-01) / 7.83E-01 (7.83E-01) eV
    forces      MAE (Best) / RMSE (Best):  3.85E-01 (3.85E-01) / 6.19E-01 (6.19E-01) eV/Ang
    dipole      MAE (Best) / RMSE (Best):  2.13E-01 (2.13E-01) / 2.77E-01 (2.77E-01) eAng



Epoch     6 |##########################################| 100.0% Done - Epoch Time:  10.2 s, Loss:  7.5870   
Epoch     7 |##########################################| 100.0% Done - Epoch Time:  10.4 s, Loss:  7.1450   
Epoch     8 |##########################################| 100.0% Done - Epoch Time:  10.1 s, Loss:  6.0065   
Epoch     9 |##########################################| 100.0% Done - Epoch Time:  8.2 s, Loss:  5.4716   
Epoch    10 |##########################################| 100.0% Done - Epoch Time:  13.1 s, Loss:  5.2265   


INFO:asparagus.src.train.tester:INFO:
Summary for test set:
  Property Metrics    MAE,       RMSE
   energy           6.53e-01,  7.43e-01 eV
   forces           2.71e-01,  4.06e-01 eV/Ang
   dipole           1.45e-01,  1.92e-01 eAng

INFO:asparagus.src.train.trainer:INFO:
Summary Epoch: 10/1000
  Loss   train / valid:  5.23E+00 / 5.44E+00  Best Loss valid: 5.44E+00
  Property Metrics (valid):
    energy      MAE (Best) / RMSE (Best):  6.39E-01 (6.39E-01) / 7.29E-01 (7.29E-01) eV
    forces      MAE (Best) / RMSE (Best):  2.96E-01 (2.96E-01) / 4.70E-01 (4.70E-01) eV/Ang
    dipole      MAE (Best) / RMSE (Best):  1.59E-01 (1.59E-01) / 2.15E-01 (2.15E-01) eAng



...

INFO:asparagus.src.train.trainer:INFO:
Summary Epoch: 995/1000
  Loss   train / valid:  1.20E-01 / 2.58E-01  Best Loss valid: 2.45E-01
  Property Metrics (valid):
    energy      MAE (Best) / RMSE (Best):  2.45E-02 (2.97E-02) / 3.61E-02 (3.82E-02) eV
    forces      MAE (Best) / RMSE (Best):  6.66E-02 (6.39E-02) / 9.68E-02 (9.40E-02) eV/Ang
    dipole      MAE (Best) / RMSE (Best):  3.09E-02 (3.05E-02) / 4.30E-02 (4.27E-02) eAng



Epoch   996 |##########################################| 100.0% Done - Epoch Time:  9.5 s, Loss:  0.1141   
Epoch   997 |##########################################| 100.0% Done - Epoch Time:  8.9 s, Loss:  0.1110   
Epoch   998 |##########################################| 100.0% Done - Epoch Time:  8.1 s, Loss:  0.1116   


INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Global parameter configuration update of 'sn2_config.json'.
Current configuration entries will be overwritten in case of conflicting entries.

INFO:asparagus.src.settings.config:INFO:
Conflict! Overwrite parameter 'data_container'.
Conflict! Overwrite parameter 'test_datasets'.
Conflict! Overwrite parameter 'test_properties'.
Conflict! Overwrite parameter 'test_store_neighbor_list'.



Epoch   999 |##########################################| 100.0% Done - Epoch Time:  11.1 s, Loss:  0.1129   


INFO:asparagus.src.train.tester:INFO:
Summary for train set:
  Property Metrics    MAE,       RMSE
   energy           2.81e-02,  3.43e-02 eV
   forces           4.42e-02,  6.16e-02 eV/Ang
   dipole           2.65e-02,  3.63e-02 eAng

INFO:asparagus.src.train.tester:INFO:
Summary for valid set:
  Property Metrics    MAE,       RMSE
   energy           2.97e-02,  3.82e-02 eV
   forces           6.39e-02,  9.40e-02 eV/Ang
   dipole           3.05e-02,  4.27e-02 eAng

INFO:asparagus.src.train.tester:INFO:
Summary for test set:
  Property Metrics    MAE,       RMSE
   energy           2.81e-02,  3.34e-02 eV
   forces           6.19e-02,  9.30e-02 eV/Ang
   dipole           3.01e-02,  4.03e-02 eAng

